In [ ]:
#import the librairies
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
import sys


In [ ]:
debug = True #activate if you want to compute the print/test to see if each step is correctly working
batch_size = 64
block_size = 64
max_iters = 5000
eval_interval = 200
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.cuda.is_available())
eval_iters = 200
n_embed = 256 #should be divible by num_head
num_heads = 8
num_blocks = 6
dropout = 0.4
fraction_training_data = 0.9


torch.manual_seed(1337)

True


### Download the data


In [ ]:
!wget https://raw.githubusercontent.com/amisha-jodhani/text-generator-harry-potter/master/1SorcerersStone.txt #Shakespeare Dataset

--2023-09-28 13:31:31--  https://raw.githubusercontent.com/amisha-jodhani/text-generator-harry-potter/master/1SorcerersStone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444307 (434K) [text/plain]
Saving to: ‘1SorcerersStone.txt’

1SorcerersStone.txt 100%[===================>] 433.89K  --.-KB/s    in 0.04s   

2023-09-28 13:31:32 (11.9 MB/s) - ‘1SorcerersStone.txt’ saved [444307/444307]



In [ ]:
with open('/content/harry-potter-7.txt', 'r', encoding='utf-8') as f:
    teext = f.read()

In [ ]:
print("length of dataset in characters: ", len(teext))

length of dataset in characters:  1346803


In [ ]:
import re

def tokenize_text(text):
    # Use a regular expression to tokenize the text into words while preserving spaces and line breaks
    tokens =  re.findall(r'\w+|[\.,;!?"]+|\n|\t|.', text)
    return tokens

# Example usage

text = tokenize_text(teext)


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'J', '.', 'K', '.', 'ROWLING', '\n', '\n', '\n', '\n', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'Harry', ' ', 'Potter', '\n', 'et', ' ', 'les', ' ', 'Reliques', ' ', 'de', ' ', 'la', ' ', 'Mort', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'Traduit', ' ', 'de', ' ', 'l', '’', 'anglais', '\n', ' ', ' ', ' ', 'par', ' ', 'Jean', '-', 'François', ' ', 'Ménard', '\n', '\n', '\n', '\n', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '2007', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'GALLIMARD', '\n', '\x0c', 'Ce', ' ', 'volume', ' ', 'est', ' ', 'dédicacé', ' ', 'à', ' ', 'sept', ' ', 'personnes', ':', '\n', '\n', '\n', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'À', ' ', 'Neil', '\n', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 

### Create a map between characters and integers
**Objective**: associate to each characters to an integers using dictionnaries

In [ ]:
import re

def tokenize_text(text):
    # Utiliser une expression régulière pour découper le texte en mots tout en conservant les espaces, la ponctuation, les guillemets, les sauts de ligne et les tabulations
    tokens = re.findall(r'\w+|[\.,;!?"]+|\n\n\t|\n|\t|.', text)
    return tokens

# Exemple d'utilisation
example_text = 'This is an example text.\n\n\tIt contains "quotes" and multiple lines!\nAnother line here.'
tokenized_words = tokenize_text(example_text)
print(tokenized_words)

['This', ' ', 'is', ' ', 'an', ' ', 'example', ' ', 'text', '.', '\n\n\t', 'It', ' ', 'contains', ' ', '"', 'quotes', '"', ' ', 'and', ' ', 'multiple', ' ', 'lines', '!', '\n', 'Another', ' ', 'line', ' ', 'here', '.']


In [ ]:
list_char = sorted(list(set(text)))
vocab_size = len(list_char)
if debug:
  print("vocab_size :", vocab_size)
  print(list_char)

vocab_size : 15415
['\n', '\x0c', ' ', '!', '!,', '(', ')', ',', '-', '.', '.,', '1', '10', '11', '12', '13', '14', '15', '16', '1689', '17', '18', '19', '1945', '1960', '1981', '1er', '2', '20', '2007', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '4', '463', '5', '6', '7', '75', '8', '9', ':', ';', '?', 'A', 'AILLEURS', 'AIT', 'ALBUS', 'ALECTO', 'ALLEZ', 'ALLONS', 'APPROCHER', 'ARCTURUS', 'ARIANA', 'ARRÊTE', 'ARRÊTEZ', 'ARTHUR', 'ATTENDEZ', 'AU', 'AUPRÈS', 'AUTORISATION', 'AUX', 'Aargh', 'Ab', 'Abandonnant', 'Abasourdi', 'Abbot', 'Abelforth', 'Absolument', 'Abus', 'Accepterais', 'Acceptez', 'Accio', 'Accompagné', 'Accroupi', 'Adalbert', 'Adieu', 'Admettons', 'Affichant', 'Agence', 'Agité', 'Aguamenti', 'Ah', 'Aide', 'Aidez', 'Aidé', 'Aie', 'Ainsi', 'Ajouter', 'Al', 'Alastor', 'Albanie', 'Albert', 'Albus', 'Alderton', 'Alecto', 'Alfred', 'Alicia', 'Alignez', 'Allait', 'Aller', 'Allez', 'Allongé', 'Allons', 'Alohomora', 'Alors', '

In [ ]:
# dictionnary char -> integer
stoi = {c:i for i,c in enumerate(list_char)}
itos = {i:c for i,c in enumerate(list_char)}

if debug:
  print("stoi :", stoi)
  print("itos :", itos)

stoi : {'\n': 0, '\x0c': 1, ' ': 2, '!': 3, '!,': 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '.,': 10, '1': 11, '10': 12, '11': 13, '12': 14, '13': 15, '14': 16, '15': 17, '16': 18, '1689': 19, '17': 20, '18': 21, '19': 22, '1945': 23, '1960': 24, '1981': 25, '1er': 26, '2': 27, '20': 28, '2007': 29, '21': 30, '22': 31, '23': 32, '24': 33, '25': 34, '26': 35, '27': 36, '28': 37, '29': 38, '3': 39, '30': 40, '31': 41, '32': 42, '33': 43, '34': 44, '35': 45, '36': 46, '4': 47, '463': 48, '5': 49, '6': 50, '7': 51, '75': 52, '8': 53, '9': 54, ':': 55, ';': 56, '?': 57, 'A': 58, 'AILLEURS': 59, 'AIT': 60, 'ALBUS': 61, 'ALECTO': 62, 'ALLEZ': 63, 'ALLONS': 64, 'APPROCHER': 65, 'ARCTURUS': 66, 'ARIANA': 67, 'ARRÊTE': 68, 'ARRÊTEZ': 69, 'ARTHUR': 70, 'ATTENDEZ': 71, 'AU': 72, 'AUPRÈS': 73, 'AUTORISATION': 74, 'AUX': 75, 'Aargh': 76, 'Ab': 77, 'Abandonnant': 78, 'Abasourdi': 79, 'Abbot': 80, 'Abelforth': 81, 'Absolument': 82, 'Abus': 83, 'Accepterais': 84, 'Acceptez': 85, 'Accio': 86, 'Accompag

In [ ]:
# function to encode and decode string using dictionnaries

def encode(text, stoi = stoi):
  list_integers = []
  for c in text:
    list_integers.append(stoi.get(c))

  return list_integers


def decode(list_integers, itos=itos):
  text = []
  for i in list_integers:
    text.append(itos.get(i))

  text = ''.join(c for c in text) #delete this line if you want a list of char instead of a str
  return text









### Preparing the data and split it into training and validation set

In [ ]:
data_tok = encode(text)
if debug : print(data_tok[:100])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 902, 9, 930, 9, 1434, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 827, 2, 1343, 0, 6560, 2, 8782, 2, 1467, 2, 4978, 2, 8681, 2, 1130, 0, 2, 2, 2, 2, 2, 2, 2, 1760, 2, 4978, 2, 8680, 15411, 2437, 0, 2, 2, 2, 10167, 2, 911, 8, 740, 2, 1141, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 29, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 749, 0, 1, 330, 2]


In [ ]:

z = int(fraction_training_data*len(data_tok))
train_set = data_tok[:z]
validation_set = data_tok[z:]

if debug:
  print("train set -> length :", len(train_set))
  print(train_set[:100], "\n")
  print("validation set -> length :", len(validation_set))
  print(validation_set[:100])


train set -> length : 483298
[2, 2, 2, 2, 2, 2, 2, 2, 2, 902, 9, 930, 9, 1434, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 827, 2, 1343, 0, 6560, 2, 8782, 2, 1467, 2, 4978, 2, 8681, 2, 1130, 0, 2, 2, 2, 2, 2, 2, 2, 1760, 2, 4978, 2, 8680, 15411, 2437, 0, 2, 2, 2, 10167, 2, 911, 8, 740, 2, 1141, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 29, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 749, 0, 1, 330, 2] 

validation set -> length : 53700
[559, 2, 8794, 2, 12756, 2, 9024, 2, 9758, 7, 2, 5084, 2, 8482, 7, 2, 6560, 2, 8680, 15411, 6580, 2, 3025, 2, 8680, 15411, 6698, 2, 4978, 2, 11433, 15411, 14343, 2, 14881, 2, 11445, 0, 9976, 2, 9459, 2, 14343, 2, 8468, 2, 3222, 9, 0, 0, 15410, 2, 1840, 2, 3045, 2, 14278, 2, 3229, 2, 6781, 7, 2, 1614, 9, 2, 387, 2, 4978, 2, 13782, 2, 9243, 2, 12174, 8, 13691, 8, 7940, 7, 2, 14881, 2, 14728, 2, 3052, 2, 57, 0, 0, 559, 2, 10228, 2, 13554, 2, 8768, 2, 13957, 2]


In [ ]:
print ("block_size :", block_size)
if debug: print(train_set[:block_size+1])

block_size : 64
[2, 2, 2, 2, 2, 2, 2, 2, 2, 902, 9, 930, 9, 1434, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 827, 2, 1343, 0, 6560, 2, 8782, 2, 1467, 2, 4978, 2, 8681, 2, 1130, 0, 2, 2, 2, 2, 2, 2, 2, 1760, 2, 4978, 2, 8680, 15411, 2437, 0, 2, 2, 2, 10167, 2, 911, 8, 740]


What is important to understand is that we are going to train our model with various length of context,
ie in list of endoded word of size *block_size+1* we are going to get block_size training examples




In [ ]:
def get_batch(split): #split is either "train" or "eval"
  assert split in ["train", "eval"], "split must be 'train' or 'eval'"
  data = train_set if split == "train" else validation_set

  ix = torch.randint(0, len(data) - block_size-1, (batch_size,)) # return a tensor of shape (batch_size) with random values bitween 0 and len(data) - block_size

  x = torch.stack([torch.tensor(data[i:i + block_size]) for i in ix])
  y = torch.stack([torch.tensor(data[i + 1:i + block_size + 1]) for i in ix])

  #y = torch.clamp(y, 0, vocab_size - 1) # not supposed to be necessary but assure that the value are between 0 and vocab_size - 1

  x, y = x.to(device), y.to(device)
  return x, y


if debug:
  xb, yb = get_batch("train")
  print(xb.shape)
  print("xb:", xb)

  print("yb:", yb)

torch.Size([64, 64])
xb: tensor([[    7,     2, 14344,  ...,   871,     2, 14751],
        [    2,  4986,     2,  ...,  7957,     2,  4556],
        [    2,  4978,     2,  ...,  7904,     7,     2],
        ...,
        [  827,     2, 13034,  ..., 12249,     0,  8681],
        [14881,     2,  7409,  ...,  6773,     2,   559],
        [    2,   943,     2,  ...,  8856,     2,    55]], device='cuda:0')
yb: tensor([[    2, 14344,     2,  ...,     2, 14751,     2],
        [ 4986,     2,   983,  ...,     2,  4556,     9],
        [ 4978,     2,  8681,  ...,     7,     2,  4963],
        ...,
        [    2, 13034,     2,  ...,     0,  8681,     2],
        [    2,  7409,     2,  ...,     2,   559,     2],
        [  943,     2,   208,  ...,     2,    55,     0]], device='cuda:0')


### Bigram Model

In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  m.eval()
  for split in ["train", "eval"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = m(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  m.train()
  return out

In [ ]:
class Head(nn.Module):
  # for the forward methods :
  # input of size (batch, time-step, channels)
  # output of size (batch, time-step, head size)``

  def __init__(self, head_size): #head_size = d_k = d_v if we are used to the notation of the original paper
    super().__init__()
    self.values = nn.Linear(n_embed, head_size, bias=False)
    self.keys = nn.Linear(n_embed, head_size, bias=False)
    self.queries = nn.Linear(n_embed, head_size, bias=False)

    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)



  def forward(self, x):
    # x.shape : (batch, time-step, channels)
    B,T,C = x.shape
    key = self.keys(x) #(B, T, head_size)
    query = self.queries(x) #(B, T, head_size)
    value = self.values(x) # same
    energy= query @ key.transpose(-2, -1)
    energy = energy/(query.shape[-1]**0.5) # (B, T, T)
    energy = energy.masked_fill(self.tril[:T, :T]==0, float("-inf"))

    energy = F.softmax(energy, dim=-1)
    energy = self.dropout(energy)
    out = energy @ value # (B, T, C)
    return out





In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range (num_heads)])
    self.proj = nn.Linear(n_embed, n_embed)
    self.dropout = nn.Dropout(dropout)



  def forward(self,x):
    x = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.proj(x)
    out = self.dropout(out)
    return out

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, n_embed):
    super().__init__()
    self.linear = nn.Linear(n_embed, 4*n_embed)
    self.linear2 = nn.Linear(4*n_embed, n_embed)
    self.dropout = nn.Dropout(dropout)
  def forward(self, x):
    x = self.linear(x)
    x = F.relu(x)
    x = self.linear2(x)
    x = self.dropout(x)


    return x

In [ ]:
class Block(nn.Module):
  def __init__(self, n_embed, num_head):
    super().__init__()
    head_size = n_embed//num_head
    self.sa = MultiHeadAttention(num_heads, head_size)
    self.ffn = FeedForward(n_embed)
    self.ln1 = nn.LayerNorm(n_embed)
    self.ln2 = nn.LayerNorm(n_embed)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffn(self.ln2(x))

    return x

In [ ]:
class BigramModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding = nn.Embedding(vocab_size, n_embed)
    self.position_embedding = nn.Embedding(block_size, n_embed)
    self.attention_blocks = nn.Sequential(
        *[Block(n_embed, num_heads) for _ in range(num_blocks)],
        nn.LayerNorm(n_embed),
    )
    self.linear = nn.Linear(n_embed,vocab_size)





  def forward(self, idx, targets=None): #idx & target are tensor integers of shape (B,T)
    B, T = idx.shape
    tok_embed = self.token_embedding(idx) # shape (B, T, C) with C = n_embed
    pos_embed = self.position_embedding(torch.arange(T, device=device)) #(T, C)
    x = pos_embed + tok_embed
    logits = self.attention_blocks(x) #(B, T, C)
    logits = self.linear(logits)#(B, T, vocab_size)





    if targets is None:
      loss =  None
    else:
    #refactoring the tensor to match the dimensions required by CrossEntropyLoss
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      #computation of the loss
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tok):
    for i in range(max_new_tok):
      idx_crop = idx[:, -block_size:] #crop
      logits, loss = self(idx_crop)

      #focus on the last time step
      logits = logits[:, -1, : ] # shape: (B, C)
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) #(B,1)

      idx= torch.cat((idx, idx_next), dim=1) #(B, T+1 )

    return idx



m = BigramModel(vocab_size)
m = m.to(device)
idx = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(idx, max_new_tok=100)[0].tolist()))








touffuesrejetteinscritDOLORES26ENQUÊTEfacilitaitmonteessaierontmatièremontaitmentonThèmedébattrechuchotadominatriceenfoncéveuxfacultébraiséviolerlégèrementpouvaientVOTREsouciaitentourèrentLorspoubellesmouvementsretrouveronsgarniechansonvitesseaccusésdressaientamplefigèrentenvoyeraffirméattendronsFlagrancesemainerêvessongercarnetsémergeaBarrrnyenleverréclamaitVraimentneabattudeboutassistaittorturéeRavitraînantenaissantechaloupéeeffleurermarionnetteobstinéeapprochaitpareillesrevêchesoulevaitexaspérantdisputesaffecterattachéesremplieanxieuxépargnerreviensAujourdagenouillerévidenttailladéprovoquéeréprouvéeprécipitepunirbusteproblèmesmatérialisaferventesdisposéesseuilduretésoutenucontrôlerchaussesvoulonsMacnairconfusionentouréensevelisvastesdresséevolume


In [ ]:
#train the model

#create the optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
for step in range (max_iters):
  xb, yb = get_batch("train")

  #evaluate the loss
  if step % eval_interval == 0 or step==max_iters-1 :
    losses = estimate_loss()
    print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['eval']:.4f}")

  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()



step 0: train loss 9.8983, val loss 9.8977
step 200: train loss 3.3261, val loss 3.3911
step 400: train loss 2.8710, val loss 3.0049
step 600: train loss 2.6163, val loss 2.8410
step 800: train loss 2.4661, val loss 2.7597
step 1000: train loss 2.3541, val loss 2.7138
step 1200: train loss 2.2561, val loss 2.7000
step 1400: train loss 2.1679, val loss 2.6823
step 1600: train loss 2.1017, val loss 2.6983
step 1800: train loss 2.0288, val loss 2.6994


KeyboardInterrupt: ignored

In [ ]:
m.eval()
print(decode(m.generate(idx, max_new_tok=5000)[0].tolist()))


du hall s’une fut instantané. La biche contempla la robe de son temps d’Ariana et lut une
telle des effets immenses en place, dans le jardin avaient presque été par endroits.

Elle tendit le premier en arrière. Les prisonniers se penchèrent dans le cuvette, ils annulèrent les rideaux de chaque
nerf d’avoir apporté le corps. Il y avait une carte de souris, pâle caché par la
fenêtre la vie où une flamme se trouvait à un obstacle et au-
dessus des têtes, et celles du désir de vomissures. Le gobelin se précipita tendu dans la pénombre. Elle semblait étrange de voir
quelque part à un gros ministre. Lorsqu’il n’prêtait plus que plus brillante qu’il avait quitté l’épée ou le soleil était venu
de l’âme, mais une seule plaque volait, chaque coin de l’accompagnait resta immobile, se tourna vers le plafond, il lui rendit la blessure
d’or ! la photo était derrière la balustrade, en lui fut atrocement en arrière-bas.

— Eh bien, ils vont insister avec
moi, nous sommes rencontrés dans l’interview. 